In [117]:
import gmpy2

# Extended Euclidean Algorithm to find the modular inverse
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1
    gcd, x1, y1 = extended_gcd(b % a, a)
    x = y1 - (b // a) * x1
    y = x1
    return gcd, x, y

def mod_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p)
    if gcd != 1:
        raise ValueError("Inverse doesn't exist")
    return x % p

# Baby-Step Giant-Step to compute discrete log
def baby_step_giant_step(g, x, p, d):
    m = int(gmpy2.isqrt(d)) + 1
    baby_steps = {}

    # Baby steps
    current = 1
    for j in range(m):
        baby_steps[current] = j
        current = (current * g) % p

    # Giant steps
    g_m_inv = mod_inverse(pow(g, m, p), p)
    current = x
    for i in range(m):
        if current in baby_steps:
            return i * m + baby_steps[current]
        current = (current * g_m_inv) % p
        
    raise ValueError("Logarithm not found")





In [118]:
baby_step_giant_step(3, 7, 17,16)

11

In [84]:
mod_inverse(11, 14)   

9

In [140]:
def pohlig_hellman(g, x, p, e, P):
    """
    Solves the discrete logarithm problem g^k = x (mod P) where the
    order of g is p^e.

    Args:
        g: The base.
        x: The target value.
        p: The prime factor of the order.
        e: The exponent of the prime factor in the order.
        P: The modulus.

    Returns:
        The discrete logarithm k, such that g^k = x (mod P).
    """
    h = pow(g, (P-1) // p, P) # or pow(g, p**(e-1), P)  # Order of h is p. (This is crucial!)
    a_list = []
    xi = x
    inv_g = mod_inverse(g, P)

    for i in range(e):
        # Find a_i such that h^(a_i) = xi^(N / p^(i+1)) (mod P) where N = p^e
        xi_reduced = pow(xi, (P-1) // (p**(i+1)), P) # or xi_reduced = pow(xi, p**(e-(i+1)), P)
        ai = baby_step_giant_step(h, xi_reduced, P, p) #dlog_h(xi_reduced)

        a_list.append(ai)
        xi = (xi * pow(inv_g, ai * (p**i), P)) % P  # Update xi

    # Combine the results using the formula: k = a_0 + a_1*p + a_2*p^2 + ... + a_(e-1)*p^(e-1)
    result = sum([a * (p**i) for i, a in enumerate(a_list)])
    return result % (p**e) # Return the result modulo the order p^e

In [141]:
pohlig_hellman(3, 7, 2, 4, 17)

11

In [142]:
# Example usage
p = 2**4 + 1  
e = 4  
g = 3  
x = 7  


dlog_result = pohlig_hellman(g, x, p, e)
print("The discrete logarithm dlog_g x is:", dlog_result)

TypeError: pohlig_hellman() missing 1 required positional argument: 'P'

In [157]:
def Chinese_Remainder_Theorem(a, b, n, m):
  #u, v, d = Extended_Euclidean(a*n, b*m)
    d, u, v = extended_gcd(n, m)
    if d == 1:
        print("x = ", (b*u*n + a*v*m)% (m*n), "mod", m*n)
        return (b*u*n + a*v*m)% (m*n), m*n
    return



In [158]:
Chinese_Remainder_Theorem(1, 1, 2, 3)

x =  1 mod 6


(1, 6)

In [161]:
from sympy import factorint

def pohlig_hellman_gen(g, p, x):
    factors = factorint(p - 1)
    print(factors)
    m = [] 
    q = []
    
    for qi, k in factors.items():
        print(qi)
        print(k)
        q.append(qi ** k)
        mi = pohlig_hellman(g, x, qi, k, p)
        print(mi)
        m.append(mi)

    if len(m) == 1:
        return m[0], q[0]  # If there's only one element, return it directly

    initm = m[0]
    initq = q[0]
    
    for i in range(1, len(m)):
        # Ensure Chinese_Remainder_Theorem returns a tuple
        result = Chinese_Remainder_Theorem(initm, m[i], initq, q[i])
        if result is None:
            raise ValueError("Chinese_Remainder_Theorem returned None, check your implementation.")
        initm, initq = result  # Unpack the result directly

    return initm, initq

In [190]:
#from sympy import mod_inverse

def elgamal_decrypt(ciphertext, private_key, p):
    """
    Decrypt an ElGamal ciphertext using the private key.
    """
    decrypted_message = []
    # If private_key is a tuple, extract the actual integer private key
    if isinstance(private_key, tuple):
        private_key = private_key[0]  # Assuming the first element is the correct key

    for c1, c2 in ciphertext:
        s = pow(c1, private_key, p)
        s_inv = mod_inverse(s, p)
        m = (c2 * s_inv) % p
        decrypted_message.append(m)
    return decrypted_message

###########################################################################
# Message Decoding Functions #
###########################################################################

# Mapping for num to char for decoding
num_to_char = {
    11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F',
    17: 'G', 18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L',
    23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R',
    29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'X',
    35: 'Y', 36: 'Z', 41: ' ', 42: "'", 43: '.', 44: ',',
    45: '?'
}

def decode_number(number):
    """
    Decode a single number into a string using the num_to_char mapping.
    """
    num_str = str(number)
    # Split the number into pairs of digits
    pairs = [int(num_str[i:i+2]) for i in range(0, len(num_str), 2)]
    # Map each pair to a character using num_to_char
    return ''.join(num_to_char.get(pair, '?') for pair in pairs)

def decode_message(decrypted_numbers):
    """
    Decode a list of decrypted numbers into a readable message.
    """
    decoded_text = ''.join(decode_number(num) for num in decrypted_numbers)
    return decoded_text

###########################################################################
# Main Execution Block #
###########################################################################

# Parameters
p = 123456789987654353003
g = 123456789
g_B = 39318628345168608817

# Ciphertext
ciphertext = [
    [83025882561049910713, 66740266984208729661],
    [117087132399404660932, 44242256035307267278],
    [67508282043396028407, 77559274822593376192],
    [60938739831689454113, 14528504156719159785],
    [5059840044561914427, 59498668430421643612],
    [92232942954165956522, 105988641027327945219],
    [97102226574752360229, 46166643538418294423]
]

# Find Bob's private key using Pohlig-Hellman
private_key = pohlig_hellman_gen(g, p, g_B)
print("=" * 80)
print(f"Bob's private key: {private_key}")
print("=" * 80)

# Decrypt the message
decrypted_message = elgamal_decrypt(ciphertext, private_key, p)
print(f"Decrypted numerical message: {decrypted_message}")
print("=" * 80)

# Decode the numerical message to text
decoded_text = decode_message(decrypted_message)
print(f"Decoded text: {decoded_text}")
print("=" * 80)

{2: 1, 23: 1, 1907: 1, 1407364059046241: 1}
2
1
1
23
1
16
1907
1
1377
1407364059046241
1
5191
x =  39 mod 46
x =  5191 mod 87722
x =  5191 mod 123456789987654353002
Bob's private key: (5191, 123456789987654353002)
Decrypted numerical message: [19244117112225192941, 16191522142944411631, 22224125164116222533, 15282944412628192319, 30193215411522152315, 24302941141124131541, 16252841182531282943]
Decoded text: IN GALOIS FIELDS, FULL OF FLOWERS, PRIMITIVE ELEMENTS DANCE FOR HOURS.


In [191]:
import gmpy2
from sympy import factorint


# Extended Euclidean Algorithm: Finds GCD(a, b) and coefficients x, y such that ax + by = GCD(a, b)
def extended_gcd(a, b):
    if a == 0:
        return b, 0, 1  # Base case: GCD(0, b) = b, x = 0, y = 1
    gcd, x1, y1 = extended_gcd(b % a, a) # Recursive call
    x = y1 - (b // a) * x1 # Calculate x
    y = x1 # Calculate y
    return gcd, x, y

# Modular Inverse: Finds the inverse of 'a' modulo 'p' (a^-1 mod p)
def mod_inverse(a, p):
    gcd, x, _ = extended_gcd(a, p) # Use extended GCD
    if gcd != 1:
        raise ValueError("Inverse doesn't exist") # a and p must be coprime
    return x % p # Return the inverse

# Baby-Step Giant-Step: Computes the discrete logarithm (log_g x mod p)
def baby_step_giant_step(g, x, p, d):
    m = int(gmpy2.isqrt(d)) + 1  # Size of baby steps/giant steps
    baby_steps = {}

    # Baby steps: Store g^j mod p for j = 0 to m-1
    current = 1
    for j in range(m):
        baby_steps[current] = j  # Store g^j and its exponent j
        current = (current * g) % p

    # Giant steps: Search for x * (g^-m)^i mod p in baby steps
    g_m_inv = mod_inverse(pow(g, m, p), p) # Precompute g^-m mod p
    current = x
    for i in range(m):
        if current in baby_steps: # Check if we have a match
            return i * m + baby_steps[current] # Return the discrete log
        current = (current * g_m_inv) % p # Calculate next giant step

    raise ValueError("Logarithm not found") # No solution


def pohlig_hellman(g, x, p, e, P):
    """
    Solves the discrete logarithm problem g^k = x (mod P) where the
    order of g is p^e.

    Args:
        g: The base.
        x: The target value.
        p: The prime factor of the order.
        e: The exponent of the prime factor in the order.
        P: The modulus.

    Returns:
        The discrete logarithm k, such that g^k = x (mod P).
    """
    h = pow(g, (P-1) // p, P) # Reduced base: order of h is p
    a_list = []
    xi = x
    inv_g = mod_inverse(g, P)

    for i in range(e):
        # Find a_i such that h^(a_i) = xi^(N / p^(i+1)) (mod P)
        xi_reduced = pow(xi, (P-1) // (p**(i+1)), P)
        ai = baby_step_giant_step(h, xi_reduced, P, p)

        a_list.append(ai)
        xi = (xi * pow(inv_g, ai * (p**i), P)) % P  # Update xi

    # Combine the results k = a_0 + a_1*p + a_2*p^2 + ... + a_(e-1)*p^(e-1)
    result = sum([a * (p**i) for i, a in enumerate(a_list)])
    return result % (p**e) # Return result modulo p^e


def Chinese_Remainder_Theorem(a, b, n, m):
    """
    Solves the system of congruences:
    x = a (mod n)
    x = b (mod m)
    """
    d, u, v = extended_gcd(n, m)
    if d == 1:  # Check if n and m are coprime
        print("x = ", (b*u*n + a*v*m)% (m*n), "mod", m*n) # Print intermediate result
        return (b*u*n + a*v*m)% (m*n), m*n # Return the solution and the new modulus
    return # No solution if n and m are not coprime


def pohlig_hellman_gen(g, p, x):
    """
    Solve discrete logarithm problem using Pohlig-Hellman algorithm.

    Args:
        g: The base.
        p: The modulus.
        x: The target value.

    Returns:
        The discrete logarithm k, such that g^k = x (mod p).
    """
    factors = factorint(p - 1) # Factorize the order of the group (p-1)
    print(factors)
    m = []
    q = []

    for qi, k in factors.items(): # Iterate through prime factors and their exponents
        print(qi)
        print(k)
        q.append(qi ** k) # Store prime power
        mi = pohlig_hellman(g, x, qi, k, p) # Compute discrete log modulo prime power
        print(mi)
        m.append(mi)

    if len(m) == 1: # Only one prime factor
        return m[0], q[0] # Return directly

    initm = m[0]
    initq = q[0]

    for i in range(1, len(m)): # Use CRT to combine solutions
        result = Chinese_Remainder_Theorem(initm, m[i], initq, q[i])
        if result is None:
            raise ValueError("Chinese_Remainder_Theorem returned None, check your implementation.")
        initm, initq = result # Update with the new solution and modulus

    return initm, initq # Return the final result and modulus

In [ ]:
def elgamal_decrypt(ciphertext, private_key, p):
    """
    Decrypts an ElGamal ciphertext using the provided private key.
    
    Parameters:
        ciphertext (list): A list of tuples containing the ciphertext components (c1, c2).
        private_key (int or tuple): The private key for decryption.
        p (int): The prime modulus.
        
    Returns:
        list: The decrypted numerical message.
    """
    decrypted_message = []
    
    # Extract the actual integer private key if it's a tuple
    if isinstance(private_key, tuple):
        private_key = private_key[0]  # Use the first element

    for c1, c2 in ciphertext:
        s = pow(c1, private_key, p)  # Compute shared secret
        s_inv = pow(s, p - 2, p)      # Compute modular inverse using Fermat's Little Theorem
        m = (c2 * s_inv) % p           # Recover the original message
        decrypted_message.append(m)
    
    return decrypted_message

###########################################################################
# Message Decoding Functions #
###########################################################################

# Mapping from numbers to characters for decoding
num_to_char = {
    11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F',
    17: 'G', 18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L',
    23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R',
    29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'X',
    35: 'Y', 36: 'Z', 41: ' ', 42: "'", 43: '.', 44: ',',
    45: '?'
}

def decode_number(number):
    """
    Decodes a single number into a string using the num_to_char mapping.
    
    Parameters:
        number (int): The number to decode.
        
    Returns:
        str: The decoded string.
    """
    num_str = str(number)
    # Split the number into pairs of digits
    pairs = [int(num_str[i:i + 2]) for i in range(0, len(num_str), 2)]
    # Convert pairs to characters
    return ''.join(num_to_char.get(pair, '?') for pair in pairs)

def decode_message(decrypted_numbers):
    """
    Decodes a list of decrypted numbers into a readable message.
    
    Parameters:
        decrypted_numbers (list): List of decrypted numbers.
        
    Returns:
        str: The fully decoded message.
    """
    return ''.join(decode_number(num) for num in decrypted_numbers)

###########################################################################
# Main Execution Block #
###########################################################################

# Parameters
p = 123456789987654353003
g = 123456789
g_B = 39318628345168608817

# Ciphertext
ciphertext = [
    [83025882561049910713, 66740266984208729661],
    [117087132399404660932, 44242256035307267278],
    [67508282043396028407, 77559274822593376192],
    [60938739831689454113, 14528504156719159785],
    [5059840044561914427, 59498668430421643612],
    [92232942954165956522, 105988641027327945219],
    [97102226574752360229, 46166643538418294423]
]

# Find Bob's private key using the Pohlig-Hellman algorithm
private_key = pohlig_hellman_gen(g, p, g_B)
print("=" * 80)
print(f"Bob's private key: {private_key}")
print("=" * 80)

# Decrypt the message
decrypted_message = elgamal_decrypt(ciphertext, private_key, p)
print(f"Decrypted numerical message: {decrypted_message}")
print("=" * 80)

# Decode the numerical message to text
decoded_text = decode_message(decrypted_message)
print(f"Decoded text: {decoded_text}")
print("=" * 80)

In [186]:
def elgamal_decrypt(ciphertext, private_key, p, g):  # Add g as parameter
    """Decrypts an ElGamal ciphertext (y, A) given the private key, modulus, and generator."""
    y, A = ciphertext  #Unpack ciphertext
    k = pow(A, private_key, p)
    k_inv = mod_inverse(k, p)
    if k_inv is None:
        raise ValueError("Modular inverse does not exist during decryption")
    z = (y * k_inv) % p
    return z

# Given values
p = 123456789987654353003
g = 123456789
gA = 52808579942366933355
gB = 39318628345168608817

ciphertext_pairs = [
    [83025882561049910713, 66740266984208729661],
    [117087132399404660932, 44242256035307267278],
    [67508282043396028407, 77559274822593376192],
    [60938739831689454113, 14528504156719159785],
    [5059840044561914427, 59498668430421643612],
    [92232942954165956522, 105988641027327945219],
    [97102226574752360229, 46166643538418294423]
]

# 1. Find Bob's private key (b) using Pohlig-Hellman with Bob's public key (gB) and g.
b, _ = pohlig_hellman_gen(g, p, gB)
print(f"Bob's private key (b): {b}")

# 2. Decrypt each ciphertext pair and map to letters directly
letter_mapping = {
    11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F', 17: 'G',
    18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L', 23: 'M', 24: 'N',
    25: 'O', 26: 'P', 27: 'Q', 28: 'R', 29: 'S', 30: 'T', 31: 'U',
    32: 'V', 33: 'W', 34: 'X', 35: 'Y', 36: 'Z', 41: ' ',
    42: '0', 43: '.', 44: ',', 45: '?'
}

message = ""
for ciphertext in ciphertext_pairs:
    plaintext_val = elgamal_decrypt(ciphertext, b, p, g) # include g as parameter
    print(f"Decrypted Value: {plaintext_val}")
    letter = letter_mapping.get(plaintext_val, "?")
    message += letter

print(f"Decrypted message: {message}")

IndentationError: expected an indented block (<ipython-input-186-b49056c6a79f>, line 6)

In [180]:
###########################################################################
# Modular Arithmetic Functions #
###########################################################################

def mod_exp(base, exp, modulus):
"""
Perform modular exponentiation: (base^exp) % modulus.
"""
if modulus == 1:
return 0
result = 1
base = base % modulus
while exp > 0:
if exp % 2 == 1:
result = (result * base) % modulus
base = (base * base) % modulus
exp //= 2
return result

def mod_inverse(a, modulus):
"""
Compute the modular inverse of a modulo modulus using the extended Euclidean algorithm.
"""
def extended_gcd(a, b):
if a == 0:
return b, 0, 1
gcd, x1, y1 = extended_gcd(b % a, a)
x = y1 - (b // a) * x1
y = x1
return gcd, x, y
gcd, x, _ = extended_gcd(a, modulus)
if gcd != 1:
raise ValueError("Modular inverse does not exist")
return (x % modulus + modulus) % modulus

###########################################################################
# Pohlig-Hellman Algorithm #
###########################################################################

def pohlig_hellman(g, h, p, q):
"""
Solve the discrete logarithm problem using the Pohlig-Hellman algorithm.
"""
factors = []
temp = q
i = 2
while i * i <= temp:
if temp % i == 0:
factors.append(i)
while temp % i == 0:
temp //= i
i += 1
if temp > 1:
factors.append(temp)

dlogs = []
for factor in factors:
gi = mod_exp(g, q // factor, p)
hi = mod_exp(h, q // factor, p)
c = 1
while c < p:
if mod_exp(gi, c, p) % p == hi % p:
dlogs.append(c)
break
c += 1

def solve_crt(dlogs, factors):
"""
Solve the Chinese Remainder Theorem (CRT) for the given dlogs and factors.
"""
n = 1
for i in factors:
n *= i
x = 0
for i in range(len(dlogs)):
yi = n // factors[i]
zi = mod_inverse(yi, factors[i])
x += dlogs[i] * yi * zi
return x % n

return solve_crt(dlogs, factors)

###########################################################################
# ElGamal Decryption #
###########################################################################

def elgamal_decrypt(ciphertext, private_key, p):
"""
Decrypt an ElGamal ciphertext using the private key.
"""
decrypted_message = []
for c1, c2 in ciphertext:
s = mod_exp(c1, private_key, p)
s_inv = mod_inverse(s, p)
m = (c2 * s_inv) % p
decrypted_message.append(m)
return decrypted_message

###########################################################################
# Message Decoding Functions #
###########################################################################

# Mapping for num to char for decoding
num_to_char = {
11: 'A', 12: 'B', 13: 'C', 14: 'D', 15: 'E', 16: 'F',
17: 'G', 18: 'H', 19: 'I', 20: 'J', 21: 'K', 22: 'L',
23: 'M', 24: 'N', 25: 'O', 26: 'P', 27: 'Q', 28: 'R',
29: 'S', 30: 'T', 31: 'U', 32: 'V', 33: 'W', 34: 'X',
35: 'Y', 36: 'Z', 41: ' ', 42: "'", 43: '.', 44: ',',
45: '?'
}

def decode_number(number):
"""
Decode a single number into a string using the num_to_char mapping.
"""
num_str = str(number)
# Split the number into pairs of digits
pairs = [int(num_str[i:i+2]) for i in range(0, len(num_str), 2)]
# Map each pair to a character using num_to_char
return ''.join(num_to_char.get(pair, '?') for pair in pairs)

def decode_message(decrypted_numbers):
"""
Decode a list of decrypted numbers into a readable message.
"""
decoded_text = ''.join(decode_number(num) for num in decrypted_numbers)
return decoded_text

###########################################################################
# Main Execution Block #
###########################################################################

# Parameters
p = 123456789987654353003
g = 123456789
g_B = 39318628345168608817

# Ciphertext
ciphertext = [
[83025882561049910713, 66740266984208729661],
[117087132399404660932, 44242256035307267278],
[67508282043396028407, 77559274822593376192],
[60938739831689454113, 14528504156719159785],
[5059840044561914427, 59498668430421643612],
[92232942954165956522, 105988641027327945219],
[97102226574752360229, 46166643538418294423]
]

# Find Bob's private key using Pohlig-Hellman
private_key = pohlig_hellman(g, g_B, p, p-1)
print("=" * 80)
print(f"Bob's private key: {private_key}")
print("=" * 80)

# Decrypt the message
decrypted_message = elgamal_decrypt(ciphertext, private_key, p)
print(f"Decrypted numerical message: {decrypted_message}")
print("=" * 80)

# Decode the numerical message to text
decoded_text = decode_message(decrypted_message)
print(f"Decoded text: {decoded_text}")
print("=" * 80)


{2: 1, 23: 1, 1907: 1, 1407364059046241: 1}
2
1
1
23
1
16
1907
1
1377
1407364059046241
1
5191
x =  39 mod 46
x =  5191 mod 87722
x =  5191 mod 123456789987654353002
Bob's private key (b): 5191
Decrypted message: ???????


In [163]:
pohlig_hellman_gen(7, ((2**7)*(3**7)*(5**2)*11 * 13)+1, 6)

{2: 7, 3: 7, 5: 2, 11: 1, 13: 1}
2
7
100
3
7
1528
5
2
1
11
1
10
13
1
2
x =  150244 mod 279936
x =  3509476 mod 6998400
x =  66495076 mod 76982400
x =  374424676 mod 1000771200


(374424676, 1000771200)

In [164]:
pohlig_hellman_gen(2, 2*3*5*7+1, 10)

{2: 1, 3: 1, 5: 1, 7: 1}
2
1
1
3
1
1
5
1
3
7
1
0
x =  1 mod 6
x =  13 mod 30
x =  133 mod 210


(133, 210)

In [ ]:
pohlig_hellman_gen(g, x, p)

In [3]:
import random

def generator(p, fact):
    while True:
        g = random.randint(2, p - 1)  # Choose a random candidate g
        
        # Check if g^(p-1)/f is congruent to 1 mod p for all factors f
        if not all(pow(g, (p - 1) // f, p) == 1 for f in fact):
            return g  # Found a valid generator



In [5]:
# Example usage
p1 = 8836229399223340013095647802612004929452496763270039906112688880393042660857048768180580202708989666003495215047944282082281151585255618828221042735510261671384102412848721487446438793136909595200642190730921610911199593203546138400567354121452180148168251884619091701775887271604413598850098806328635477629513139087502706762929259830544462377995901
p2 = 92859401349195800914419340000027009681568723801475793017015602958169754059993011392652944281186250974103044906776823054555505968029620083486871686062658793692019452353380264379257810627455061341286362327958758493917072386515923960302537595419398183346193327838222062789049727171559396156231251823251982706595211633733555873147915114928658181413582003
p = 1641053944392086841921453321058207950444404193817738819473219740270491539459431794790179333198029370519198253827726748664830869236760835903417691319451047305414486954875153737239972475561536800287976368891755115763871510113547756663208548832274327268127263128729961208941625385185852405053532100044643990538296382247743234839833431437598837803711535954141156563558276004799497156738553666297843661349842435716824481757329925850674456792218265058027918615626063402333521893362715365180009430433836026547263073941949585226085934525247984227001466760694409050266285818147599806051959885649489654955078463497446671054793009320004149656371232004170389111728473163917246760060531775088682357931537722739407
fact = [2, p1, p2]  # Factors of p-1 (22 = 2 * 11)

g = generator(p, fact)
print("Generator:", g)

Generator: 1084747730452845696068647849612308304348808349656409277542384121006130835155308761153169499563143752081131753244044887092773271503698984932106671891054906106132540564238933244137032944331492747714909427621197642601713725183021238952151082387170943855198461804277182067718680708004726076531358926292902501712613473496781991397547958571075367244000063206762482716114495543717424144200530885513196994861390915082794137310121341396401178279266723235723438225929595245941992350835052564473545169781792257835609218426538555158269156183997706197348522319650394148609663282590145672675091350925241128254889462681843510729682561824593941368476269056838685400312603384522545123506291586795695181862124131595520


In [6]:
d= p-1
b = random.randint(2, d - 1)
B = pow(g, b, p)
B

1110029018361296539066366592365687201764819350405670713296964539682326223356665750484805543086916027547726162412314714549582560315011209677022807412421353899657564275801152623328849406869614617373613020680687263254651540858894915097191239724369967807071077062928577470566068224078939634092102313093877312045803907632704205111719203397382022501070770521246247865387658028531295171063258564003653917842091364500701038790067530411061298870785789362931105372641334407989226588621777603030701060514501025631276410107816624490641970266249452442768673650059720900838355454171810957920200739892568677480229825815149366951004661084305744555687988995494140062667542040740475681752619709189076140149324272332899

In [201]:
import random
from sympy import factorint

def sieve_of_eratosthenes(limit):
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    for i in range(2, int(limit ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, limit + 1, i):
                sieve[j] = False
    return [i for i in range(limit + 1) if sieve[i]]

def miller_rabin(n, k=5):
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False
    r, s = n - 1, 0
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = (x * x) % n
            if x == n - 1:
                break
        else:
            return False
    return True

def find_nth_prime(n, initial_limit=10000):
    # Step 1: Use Sieve to get initial primes
    primes = sieve_of_eratosthenes(initial_limit)
    count = len(primes)
    
    # If n is within the initial sieve range
    if count >= n:
        return primes[n - 1]
    
    # Step 2: Extend using Miller-Rabin
    num = initial_limit + 1
    while count < n:
        if miller_rabin(num):
            count += 1
            if count == n:
                return num
        num += 1
    return None  # Should not reach here if n is valid

# Part 1: Find the (5000000 + n)-th prime
# Replace n with your birthday in mmddyy format (e.g., 030725 for March 7, 2025)
n = 62702  # Example: March 7, 2025 (030725)
k = 5000000 + n  # The index of the prime we want

# Estimate upper bound using Prime Number Theorem
import math
upper_bound_estimate = int(k * math.log(k) * 1.1)  # 10% buffer
print(f"Estimated upper bound for the {k}-th prime: {upper_bound_estimate}")

# Find the k-th prime (this is a simplified approach; for large k, use segmented sieve or library)
# Note: Directly sieving to 80M+ is impractical here; this is a demo implementation
kth_prime = find_nth_prime(k)
print(f"The {k}-th prime number is: {kth_prime}")

# Part 2: Count primes between 225 and 226 using the sieve
limit = 226  # Small enough to sieve directly
primes = sieve_of_eratosthenes(limit)
primes_in_range = [p for p in primes if 225 <= p <= 226]
num_primes = len(primes_in_range)
print(f"Number of primes between 225 and 226: {num_primes}")

# Verify with factorization for confirmation
print(f"Factorization of 225: {factorint(225)}")  # {3: 2, 5: 2} -> composite
print(f"Factorization of 226: {factorint(226)}")  # {2: 1, 113: 1} -> composite

Estimated upper bound for the 5062702-th prime: 85970512


KeyboardInterrupt: 

In [210]:
def sieve_of_eratosthenes(limit):
    # Create a boolean array "is_prime[0..limit]" and initialize all entries as true.
    is_prime = [True] * (limit + 1)
    p = 2
    is_prime[0] = is_prime[1] = False  # 0 and 1 are not prime numbers

    while p * p <= limit:
        # If is_prime[p] is not changed, then it is a prime
        if is_prime[p]:
            # Mark all multiples of p as false
            for i in range(p * p, limit + 1, p):
                is_prime[i] = False
        p += 1

    # Collecting all prime numbers
    primes = [p for p in range(limit + 1) if is_prime[p]]
    return primes

# Example usage
limit = 100000000
primes = sieve_of_eratosthenes(limit)
print(f"Number of primes up to {limit}: {len(primes)}")

Number of primes up to 100000000: 5761455


In [215]:
import numpy as np
from math import isqrt

def sieve_of_eratosthenes(n):
    """
    Generate all prime numbers up to n using the Sieve of Eratosthenes algorithm
    Optimized for large numbers (e.g., 100 million)
    """
    # Use boolean array for memory efficiency
    sieve = np.ones(n + 1, dtype=bool)
    sieve[0] = sieve[1] = False
    
    # Only need to check up to square root of n
    for i in range(2, isqrt(n) + 1):
        if sieve[i]:
            # Mark multiples as non-prime
            # Start from i*i as all smaller multiples would have been marked
            sieve[i * i:n + 1:i] = False
    
    # Get the prime numbers
    primes = np.nonzero(sieve)[0]
    return primes


    


In [222]:

limit = 100000000
n_birthday = 62702
nth_prime = 5000000 + n_birthday
print("=========================================================")
print(f"Calculating primes up to {limit}...")
print("=========================================================")

primes = sieve_of_eratosthenes(limit)

print(f"Total number of primes found: {len(primes)}")
print("=========================================================")
print(f"The {nth_prime}, prime is: {primes[nth_prime]} ")
print("=========================================================")


Calculating primes up to 100000000...
Total number of primes found: 5761455
The 5062702, prime is: 87173869 


In [233]:
#Count primes between 2^25 and 2^26
primes_in_range = [p for p in primes if pow(2,25) < p < pow(2,26)]
print("=========================================================")
num_primes = len(primes_in_range)
print(f"Number of primes between 2^25 and 2^26 : {num_primes}")
print("=========================================================")


Number of primes between 2^25 and 2^26 : 1894120


In [ ]:
import random
from sympy import factorint

def sieve_of_eratosthenes(limit):
    sieve = [True] * (limit + 1)
    sieve[0] = sieve[1] = False
    for i in range(2, int(limit ** 0.5) + 1):
        if sieve[i]:
            for j in range(i * i, limit + 1, i):
                sieve[j] = False
    return [i for i in range(limit + 1) if sieve[i]]

def miller_rabin(n, k=5):
    if n == 2 or n == 3:
        return True
    if n < 2 or n % 2 == 0:
        return False
    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = (x * x) % n
            if x == n - 1:
                break
        else:
            return False
    return True


# 4. Use the Eratosthenes' sieve to find the (5000000 + n)-th prime number where n is your birthday (mmddyy).
# How many prime numbers do we have between 225 and 226?
def find_nth_prime(n):
    # An estimation formula is n * (ln(n) + ln(ln(n))) if n is big enough
    import math
    limit = int(n * (math.log(n) + math.log(math.log(n)))) + 100000

    count = 0
    num = 2
    while count < n:
        if miller_rabin(num):  # Use Miller-Rabin test instead of naive approach
            count += 1
        num += 1
    return num - 1  # return the nth prime number

# Use Sieve of Eratosthenes since we need to find all primes in a given range
def count_primes_between(lower_bound, upper_bound):
    primes = sieve_of_eratosthenes(upper_bound)
    count = 0
    for prime in primes:
        if lower_bound < prime <= upper_bound:
            count += 1
    return count

# Let's assume a birthday: January 1, 2000 (062702)
n_birthday = 62702
nth_prime = find_nth_prime(5000000 + n_birthday)
print(f"The {5000000 + n_birthday}th prime number is: {nth_prime}")

count_primes = count_primes_between(225, 226)
print(f"The number of primes between 225 and 226 is {count_primes}")